# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [196]:
DATA_FOLDER = 'ebola/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [276]:
# Write your answer here
import glob
import pandas as pd
import numpy as np
import os

def import_from_forder(folder_path):
    allFiles = glob.glob(folder_path + '/*.csv')
    frame = pd.DataFrame()
    list_ = []

    for file_ in allFiles:
        df = pd.read_csv(file_, header=0)
        list_.append(df)
    frame = pd.concat(list_)
    return frame

In [308]:
folders = [x[0] for x in os.walk(DATA_FOLDER)][1:]

#dataframes from 3 different folders
df1,df2,df3 = [import_from_forder(x) for x in folders]

# put date and description and total first (it was just for vision, it's useless)
df1 = df1[['Date', 'Description', 'Totals', 'Beyla', 'Boffa', 'Conakry', 'Coyah', 'Dabola', 
       'Dalaba', 'Dinguiraye', 'Dubreka', 'Forecariah', 'Gueckedou',
       'Kerouane', 'Kindia', 'Kissidougou', 'Kouroussa', 'Lola', 'Macenta',
       'Mzerekore', 'Nzerekore', 'Pita', 'Siguiri', 'Telimele', 
        'Yomou']]

df2 = df2[['Date', 'Variable', 'National', 'Bomi County', 'Bong County','Gbarpolu County', 'Grand Bassa',
       'Grand Cape Mount', 'Grand Gedeh', 'Grand Kru', 'Lofa County',
       'Margibi County', 'Maryland County', 'Montserrado County', 
       'Nimba County', 'River Gee County', 'RiverCess County', 'Sinoe County',
       'Unnamed: 18']]

df3 = df3[['date', 'variable', 'National','34 Military Hospital', 'Bo', 'Bo EMC', 'Bombali', 'Bonthe',
       'Hastings-F/Town', 'Kailahun', 'Kambia', 'Kenema', 'Kenema (IFRC)',
       'Kenema (KGH)', 'Koinadugu', 'Kono', 'Moyamba',
       'Police training School', 'Police traning School', 'Port Loko',
       'Pujehun', 'Tonkolili', 'Unnamed: 18', 'Western area',
       'Western area combined', 'Western area rural', 'Western area urban']]

# get same notation for the 3 dataframes
df1 = df1.rename(columns = {'Totals':'Totals1'})
df2 = df2.rename(columns = {'Variable':'Description', 'National':'Totals2'})
df3 = df3.rename(columns = {'date':'Date', 'variable':'Description', 'National':'Totals3'})

In [340]:
t = df1.Description.unique()

[s for s in t if all(x in s for x in ['death', 'New'])]

['New deaths registered today',
 'New deaths registered today (confirmed)',
 'New deaths registered today (probables)',
 'New deaths registered today (suspects)',
 'New deaths registered',
 'New deaths registered among health workers']

In [341]:
# take row we're interested in, new death and case
def filter_by_terms(df, terms, match=True):
    if match:
        return df.loc[df.Description.str.match('|'.join(terms))]
    else:
        return df.loc[df.Description.str.contains('|'.join(terms))]

df1 = filter_by_terms(df1, ['Total new cases registered so far', 'New deaths registered', 'New deaths registered today', 'New deaths registered among health workers'])

df2 = filter_by_terms(df2, ['Newly reported deaths', 'Newly Reported deaths in HCW, ',
                           'New Case/s (Suspected)', 'New Case/s (Probable)', 'New case/s (confirmed)'])

df3 = filter_by_terms(df3, ['new_noncase', 'etc_new_deaths', 'new_suspected', 'new_probable', 'new_confirmed',
                            'death_suspected', 'death_probable', 'death_confirmed',])

# fill NaN in totals with the sum of the row
df1["Totals1"] = df1["Totals1"].fillna(df1[df1.columns[3:]].sum(axis=1))
df2["Totals2"] = df2["Totals2"].fillna(df2[df2.columns[3:]].sum(axis=1))
df3["Totals3"] = df3["Totals3"].fillna(df3[df3.columns[3:]].sum(axis=1))

In [342]:
# concatenate the three dataframes and reorder columns
df = pd.concat([df1,df2,df3])
df = df[['Date', 'Description', 'Totals1', 'Totals2', 'Totals3']]

# cast right types
df.Date = pd.to_datetime(df.Date)
df.Totals1 = pd.to_numeric(df.Totals1)
df.Totals2 = pd.to_numeric(df.Totals2)
df.Totals3 = pd.to_numeric(df.Totals3)


Total_death = filter_by_terms(Totals, ['death', 'Death', 'deaths', 'Deaths'], False)
Total_death.groupby(Total_death.Date.dt.to_period("M")).mean().fillna("")

,Totals1,Totals2,Totals3
Date,,,
2014-06,,2.000000,
2014-07,,4.272727,
2014-08,1.58333,23.222222,0
2014-09,1.78125,36.041667,0.965517
2014-10,7.5,28.040000,3.67857
2014-11,,13.466667,0.571429
2014-12,,0.000000,2.2


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here